In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import os

In [4]:
directory = './seasons/England-Premier-League'
year = 22
month = 1
day = 22

prevthis = False
thisnew = False

games = []

for foldername in os.listdir(directory):
    if int(foldername[-2:]) < year:
        for filename in os.listdir(os.path.join(directory, foldername)):
            games.append(pd.read_csv(os.path.join(directory, foldername, filename)))
    elif int(foldername[-2:]) == year: prevthis = True
    elif int(foldername[2:4]) == year: thisnew = True

if prevthis:
    for filename in os.listdir(os.path.join(directory, '20' + str(year-1) + '-20' + str(year))):
        match_date = [int(d) for d in filename.split('-')[0:3]]
        if match_date[0] <= day and match_date[1] <= month and match_date[2] <= year:
            games.append(pd.read_csv(os.path.join(directory, '20' + str(year-1) + '-20' + str(year), filename)))


if thisnew:
    for filename in os.listdir(os.path.join(directory, '20' + str(year) + '-20' + str(year+1))):
        match_date = [int(d) for d in filename.split('-')[0:3]]
        if match_date[0] <= day and match_date[1] <= month and match_date[2] <= year:
            games.append(pd.read_csv(os.path.join(directory, '20' + str(year) + '-20' + str(year+1), filename)))

print(len(games))


41


In [ ]:
class GoalPredictor(keras.Model):
    def __init__(self):
        super().__init__()
        self.teams_advantage_predictor = createTeamAdvPred() # year, month, hour -> 2
        self.players_efficiency_predictor = createPlayerEffPred() # min, goal, assist, yellow, red, shot, pass, aer, mom, rating -> 22
        self.goal_predictor = createEffAdvGoalPred() # 11 efficiencies, 1 team advantage -> 1

    def call(self, inputs : list): # [[date], [team A data], [team B data]]
        if len(inputs) != 3: return

        advantages = self.teams_advantage_predictor(inputs[0])

        ##
        # LOAD ALL INFO FROM DB, FIND ALL GAMES BEFORE THIS GAME, FIND ALL PLAYERS STATISTICS OVERALL IN THIS GAME FROM PREVIOUS GAMES -> PEP 
        ##

        teamA = [advantages[0]]
        teamB = [advantages[1]]

        for player in inputs[1]:
            teamA(self.players_efficiency_predictor(player))
        for player in inputs[2]:
            teamB(self.players_efficiency_predictor(player))

        return [self.goal_predictor(teamA), self.goal_predictor(teamB)]
    
    def preprocess(date: list, names: list[str], teams: list[str]):
        game_data = []
        year = date[0]

        directory = './seasons/England-Premier-League'

        for foldername in os.listdir(directory):
            pass

        for name in names:
            pass



        Adet = teams[0]
        teamA = [names[i] for i in range(len(names)) if teams[i] == Adet]
        teamB = [names[i] for i in range(len(names)) if teams[i] != Adet]

        return [teamA, teamB]
    
    def train(self, X, y):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
        for index in X:
            self.train_step(X[index], y[index], optimizer)
    
    def train_step(self, X, y, optimizer):
        with tf.GradientTape() as tape:
            loss = tf.reduce_sum(tf.square(X-y))

        grad = tape.gradient(loss, self.trainable_variables)
        optimizer.apply_gradients([grad, self.trainable_variables])



def createTeamAdvPred():
    inputs = tf.keras.Input(shape=(3,))
    outputs = tf.keras.layers.Dense(2)(inputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def createPlayerEffPred():
    inputs = tf.keras.Input(shape=(10,))
    outputs = tf.keras.layers.Dense(1)(inputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def createEffAdvGoalPred():
    inputs = tf.keras.Input(shape=(12,))
    outputs = tf.keras.layers.Dense(1)(inputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs)